In [1]:
import pandas as pd
import numpy as np

df_train_b = pd.read_excel('./Train/train_bureau.xlsx',engine='openpyxl')
df_train_data = pd.read_excel('./Train/train_Data.xlsx',engine='openpyxl')



In [2]:
df_test_b = pd.read_excel('./Test/test_bureau.xlsx',engine='openpyxl')
df_test_data = pd.read_excel('./Test/test_Data.xlsx',engine='openpyxl')

In [3]:
clmn_list = ['SELF-INDICATOR', 'MATCH-TYPE', 'ACCT-TYPE', 'CONTRIBUTOR-TYPE', 
             'OWNERSHIP-IND','ACCOUNT-STATUS','INSTALLMENT-FREQUENCY','ASSET_CLASS']

df_train_b['ASSET_CLASS'] = df_train_b.groupby('ID')['ASSET_CLASS'].ffill()
# df_train_b['ASSET_CLASS'].value_counts()

df_train_b['ASSET_CLASS'].fillna('NSFD',inplace=True)
# df_train_b['ASSET_CLASS'].value_counts()

def Mrgd_df_all(lst):
    df_lst = []
    for f in lst:
        df = df_train_b.groupby('ID')[f].value_counts().unstack().add_prefix(f+'_').fillna(0)
        df_lst.append(df)

    return df_lst

#drop installment frequency and fill Asset class missing values
df_train_b.drop('INSTALLMENT-FREQUENCY',axis=1,inplace=True)
clmn_list.remove('INSTALLMENT-FREQUENCY')

result_a2 = pd.concat(Mrgd_df_all(clmn_list), join='inner', axis=1)
result_a2.reset_index(inplace=True)
result_a2.shape

(128655, 92)

In [4]:
df_train_b['INSTALLMENT-AMT'] = df_train_b['INSTALLMENT-AMT'].apply(lambda x: str(x).split('/')[0])

#'CREDIT-LIMIT/SANC AMT' ,'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT' ,'CURRENT-BAL' ,'OVERDUE-AMT', 'WRITE-OFF-AMT', 'TENURE'
# del result_b1,tl_df,df_lst
def total_ammount(cmn_nm):
    '''use only when needed'''
    df_lst=[]
    tl_df = pd.DataFrame()
    for cm in cmn_nm:
        if type(df_train_b[cm][0]) != np.float64:
            df_train_b[cm] = df_train_b[cm].str.replace(',', '').astype(float)
        tl_df= df_train_b.groupby('ID')[cm].sum()
        df_lst.append(tl_df)

    return(df_lst)

tl_clmn_list = ['CREDIT-LIMIT/SANC AMT' ,'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT' ,'CURRENT-BAL' ,'OVERDUE-AMT', 'WRITE-OFF-AMT', 'TENURE']
result_b1 = pd.concat(total_ammount(tl_clmn_list), join='inner', axis=1).add_prefix('Total_')
result_b1.reset_index(inplace=True)
result_b1.shape

bureua_data_f =pd.merge(result_a2,result_b1,how='inner',on='ID',)
bureua_data_f.shape

(128655, 99)

In [5]:
other_sums = ['DATE-REPORTED','DISBURSED-DT','CLOSE-DT','LAST-PAYMENT-DATE']
rb2= df_train_b.groupby('ID')[other_sums].count().reset_index()
# rb2.head()

rb2.rename(columns={'ID':'ID',
            'DATE-REPORTED':'Total_Times_reported',
            'DISBURSED-DT':'Total_Times_disbursed',
            'CLOSE-DT':'Total_Times_closed',
            'LAST-PAYMENT-DATE':'Total_last_payments'},inplace=True)

bureua_data_f =pd.merge(bureua_data_f,rb2,how='inner',on='ID',)
bureua_data_f.shape

(128655, 103)

In [6]:
df_train_b['REPORTED DATE - HIST']  = df_train_b['REPORTED DATE - HIST'].apply(lambda x: x.split(',') if type(x)==str else x).apply(lambda x: x[:-1] if type(x)==list else x)
df_train_b['REPORTED DATE - HIST'] = df_train_b['REPORTED DATE - HIST'].apply(lambda x: len(x) if type(x)==list else x)
rc1 = df_train_b.groupby('ID')['REPORTED DATE - HIST'].sum().reset_index()

rc1.columns = ['ID','Total_Times_Reported']
bureua_data_f =pd.merge(bureua_data_f,rc1,how='inner',on='ID',)
bureua_data_f.shape

(128655, 104)

In [7]:
def amt_sum(clm,df):
    for cl in clm:
        df[cl] = df[cl].apply(lambda x: list(filter(lambda item: item,x.split(','))) if type(x)==str else x) #remove comma
        df[cl] = df[cl].apply(lambda d: d if isinstance(d, list) else [0.0])#fill na
        df[cl] = df[cl].apply(lambda x: list(map(float,x)) if type(x)==list else x)#convert all the list elements to float
        df[cl] = df[cl].apply(lambda x:sum(x))#summ the ammount

amt_sm_cl = ['CUR BAL - HIST','AMT OVERDUE - HIST','AMT PAID - HIST']
amt_sum(amt_sm_cl,df_train_b)

rd1 = df_train_b.groupby('ID')[amt_sm_cl].sum().reset_index()
rd1.rename(columns={'ID':'ID','CUR BAL - HIST':'Total_CUR_BAL','AMT OVERDUE - HIST':'Total_Overdue_Amt','AMT PAID - HIST':'Total_Paid_Amt'})

bureua_data_f =pd.merge(bureua_data_f,rd1,how='inner',on='ID',)
bureua_data_f.shape

(128655, 107)

In [8]:
import textwrap
def for_scN_DPD(nu):
    if 'E' in nu:
        nu = '%f'%np.float64(nu)
        nu = textwrap.wrap(nu,3)
        # print(nu)
        nu =list(map(float,nu))

    return sum(nu)

#split into part of 3 digits
#check if its string
#if its not filter out keep only ints
#sum the total of list
def clean_DPD(x):
    wp = textwrap.wrap(x,3)
    if not all(str(s).isdigit() for s in wp):
        # print(wp
        wp = list(filter(lambda x: x not in ['XXX','DDD'], wp))
        wp = list(map(int, wp))
    else:
        wp = list(map(int,wp))

    return sum(wp)

# for_scN_DPD(df_train_b['DPD - HIST'][26])
df_train_b['DPD - HIST'].fillna('000',inplace=True)
df_train_b['DPD - HIST'] = df_train_b['DPD - HIST'].apply(lambda x:for_scN_DPD(x) if 'E' in x else clean_DPD(x))
rd3 = df_train_b.groupby('ID')['DPD - HIST'].sum().reset_index()

rd3.columns = ['ID','Total_DPD']
bureua_data_f =pd.merge(bureua_data_f,rd3,how='inner',on='ID',)
bureua_data_f.shape

(128655, 108)

In [9]:
print(len(result_a2.columns))
def drop_VC_BL_100(clm_nm,data_f):
    for i in df_train_b[clm_nm].value_counts().loc[lambda x : x<100].index:
        data_f.drop(clm_nm+'_%s'%i,axis=1,inplace=True)

#now for CONTRIBUTOR-TYPE_ ACCOUNT-STATUS_ ASSET_CLASS_
# for cl in ['ACCT-TYPE','CONTRIBUTOR-TYPE','ACCOUNT-STATUS','ASSET_CLASS']:
#     drop_VC_BL_100(cl,result_a2)

print(len(result_a2.columns))

for cl in ['ACCT-TYPE','CONTRIBUTOR-TYPE','ACCOUNT-STATUS','ASSET_CLASS']:
    drop_VC_BL_100(cl,bureua_data_f)

92
92


In [10]:
def mss_ct(ref_cl,ms_cl,df):
    ''' function to fill the NaN values by mode method
        Ex: in Particular State which is more frequent occured city
            same for Area in Area
                     SEX in Area
                     Age in Area
                     Salary in Area
        ref_cl : Referece Column,
        ref_nm : Element of refernce column, 
        df     : dataframe to oerform fill na operation
    
    '''
    md = df[ms_cl].mode()[0]
    frqnt_val = lambda x: x.fillna(x.value_counts().idxmax() if x.value_counts().max() >=1 else md , inplace = False)
    df[ms_cl]=df.groupby([ref_cl])[ms_cl].apply(frqnt_val)
    df[ms_cl] = df[ms_cl].fillna(df[ms_cl].value_counts().idxmax())
    # return df

In [11]:
def find_VC_St(ref_cl,ref_nm,tg):
    ''' ref_cl : Referece Column,
        ref_nm : Element of refernce column,
        tg     : column name to fill nan   '''
    vl_cm =df_train_data[df_train_data[ref_cl]==ref_nm][tg].value_counts()
#     print(vl_cm)
    re_nm_tg_cnt = (vl_cm.index[0],vl_cm[0])
    return re_nm_tg_cnt

print(find_VC_St('State','WEST BENGAL','City'),find_VC_St('State','ANDHRA PRADESH','City'))

mss_ct('State','City',df_train_data)
mss_ct('City','Area',df_train_data)
mss_ct('Area','SEX',df_train_data)
mss_ct('Area','AGE',df_train_data)
mss_ct('Area','MonthlyIncome',df_train_data)

print(find_VC_St('State','WEST BENGAL','City'),find_VC_St('State','ANDHRA PRADESH','City'))
df_train_data['MaturityDAte'].fillna(method='ffill',inplace=True)
print(any(df_train_data.isna().sum()>0))

('BARDDHAMAN', 894) ('GUNTUR', 1756)
('BARDDHAMAN', 1820) ('GUNTUR', 3172)
True


In [12]:
dro_clm = ['BranchID','AssetID','ManufacturerID','SupplierID','ZiPCODE']
df_train_data.drop(dro_clm,axis=1,inplace=True)

dt_cl = ['DisbursalDate','MaturityDAte','AuthDate']

def handel_dt(clm):
    for cl in clm:
        df_train_data[cl+'_Year'] = df_train_data[cl].dt.year.astype(float)
        df_train_data[cl+'_Month']= df_train_data[cl].dt.month.astype(float)
        df_train_data[cl+'_Week']= df_train_data[cl].dt.week.astype(float)
        df_train_data[cl+'_Day']= df_train_data[cl].dt.day.astype(float)

handel_dt(dt_cl)
df_train_data.drop(dt_cl,axis=1,inplace=True)

c:\users\vishal.nadagiri\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  # Remove the CWD from sys.path while we load stuff.


In [13]:
train_id = df_train_data['ID']
target = df_train_data['Top-up Month']

TUP_Mn = { id_en:i for i,id_en in enumerate(df_train_data['Top-up Month'].value_counts().index) }

target = target.map(TUP_Mn)

In [14]:
#since the target is Ordinal we can assign no top to 0
#since the target is Ordinal we can assign no top to 0
# TUP_Mn = { id_en:i for i,id_en in enumerate(df_train_data['Top-up Month'].value_counts().index) }

print('Before droping na',df_train_data.shape)
df_train_data.dropna(inplace=True)
print('After droping na',df_train_data.shape)


Before droping na (128655, 30)
After droping na (128655, 30)


In [15]:
#Nominal cat features 
cat_ftrs = ['Frequency' ,'InstlmentMode' ,'LoanStatus', 'PaymentMode', 'Area' ,'SEX' ,'City' ,'State']

!pip install category-encoders
from category_encoders.target_encoder import TargetEncoder

# feature_list = list(all_train.columns)
TE_encoder = TargetEncoder()
# train_te = TE_encoder.fit_transform(df_train_data[cat_ftrs], target)

for c in cat_ftrs:
    TE_encoder.fit(df_train_data[c].values,target) 
    df_train_data[c] = TE_encoder.transform(list(df_train_data[c].values))
    
for c in cat_ftrs:
    TE_encoder.fit(df_train_data[c].values,target) 
    df_test_data[c] = TE_encoder.transform(list(df_test_data[c].values))

c:\users\vishal.nadagiri\appdata\local\programs\python\python36\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\users\vishal.nadagiri\appdata\local\programs\python\python36\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\users\vishal.nadagiri\appdata\local\programs\python\python36\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\users\vishal.nadagiri\appdata\local\programs\python\python36\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed i

In [16]:
df_train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128655 entries, 0 to 128654
Data columns (total 30 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   128655 non-null  int64  
 1   Frequency            128655 non-null  float64
 2   InstlmentMode        128655 non-null  float64
 3   LoanStatus           128655 non-null  float64
 4   PaymentMode          128655 non-null  float64
 5   Area                 128655 non-null  float64
 6   Tenure               128655 non-null  int64  
 7   AssetCost            128655 non-null  int64  
 8   AmountFinance        128655 non-null  float64
 9   DisbursalAmount      128655 non-null  float64
 10  EMI                  128655 non-null  float64
 11  LTV                  128655 non-null  float64
 12  SEX                  128655 non-null  float64
 13  AGE                  128655 non-null  float64
 14  MonthlyIncome        128655 non-null  float64
 15  City             

In [34]:
df_test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14745 entries, 0 to 14744
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   14745 non-null  int64  
 1   Frequency            14745 non-null  float64
 2   InstlmentMode        14745 non-null  float64
 3   LoanStatus           14745 non-null  float64
 4   PaymentMode          14745 non-null  float64
 5   Area                 14745 non-null  float64
 6   Tenure               14745 non-null  int64  
 7   AssetCost            14745 non-null  int64  
 8   AmountFinance        14745 non-null  float64
 9   DisbursalAmount      14745 non-null  float64
 10  EMI                  14745 non-null  float64
 11  LTV                  14745 non-null  float64
 12  SEX                  14745 non-null  float64
 13  AGE                  14745 non-null  float64
 14  MonthlyIncome        14745 non-null  float64
 15  City                 14745 non-null 

In [17]:
all_train = pd.merge(bureua_data_f,df_train_data,how='inner',on='ID',)

print('Before droping na',all_train.shape)
all_train.dropna(inplace=True)
print('After droping na',all_train.shape)
all_train.drop(['ID','Top-up Month'],axis=1,inplace=True)

Before droping na (128655, 114)
After droping na (128655, 114)


# Preprocess all test data

In [18]:
# clmn_list = ['SELF-INDICATOR', 'MATCH-TYPE', 'ACCT-TYPE', 'CONTRIBUTOR-TYPE', 
#              'OWNERSHIP-IND','ACCOUNT-STATUS','INSTALLMENT-FREQUENCY','ASSET_CLASS']

df_test_b['ASSET_CLASS'] = df_test_b.groupby('ID')['ASSET_CLASS'].ffill()
# df_train_b['ASSET_CLASS'].value_counts()

df_test_b['ASSET_CLASS'].fillna('NSFD',inplace=True)
# df_train_b['ASSET_CLASS'].value_counts()

In [19]:
def Mrgd_df_all_ts(lst):
    df_lst = []
    for f in lst:
        df = df_test_b.groupby('ID')[f].value_counts().unstack().add_prefix(f+'_').fillna(0)
        df_lst.append(df)

    return df_lst

#drop installment frequency and fill Asset class missing values
df_test_b.drop('INSTALLMENT-FREQUENCY',axis=1,inplace=True)
# clmn_list.remove('INSTALLMENT-FREQUENCY')

result_a2_ts = pd.concat(Mrgd_df_all_ts(clmn_list), join='inner', axis=1)
result_a2_ts.reset_index(inplace=True)
result_a2_ts.shape

(14745, 83)

In [20]:
df_test_b['INSTALLMENT-AMT'] = df_test_b['INSTALLMENT-AMT'].apply(lambda x: str(x).split('/')[0])

#'CREDIT-LIMIT/SANC AMT' ,'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT' ,'CURRENT-BAL' ,'OVERDUE-AMT', 'WRITE-OFF-AMT', 'TENURE'
# del result_b1,tl_df,df_lst
def total_ammount(cmn_nm):
    '''use only when needed'''
    df_lst=[]
    tl_df = pd.DataFrame()
    for cm in cmn_nm:
        if type(df_test_b[cm][0]) != np.float64:
            df_test_b[cm] = df_test_b[cm].str.replace(',', '').astype(float)
        tl_df= df_test_b.groupby('ID')[cm].sum()
        df_lst.append(tl_df)

    return(df_lst)

tl_clmn_list = ['CREDIT-LIMIT/SANC AMT' ,'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT' ,'CURRENT-BAL' ,'OVERDUE-AMT', 'WRITE-OFF-AMT', 'TENURE']
result_b1_ts = pd.concat(total_ammount(tl_clmn_list), join='inner', axis=1).add_prefix('Total_')
result_b1_ts.reset_index(inplace=True)
result_b1_ts.shape


(14745, 8)

In [21]:
bureua_data_f_ts =pd.merge(result_a2_ts,result_b1_ts,how='inner',on='ID',)
bureua_data_f_ts.shape

(14745, 90)

In [22]:
other_sums = ['DATE-REPORTED','DISBURSED-DT','CLOSE-DT','LAST-PAYMENT-DATE']
rb2_ts= df_test_b.groupby('ID')[other_sums].count().reset_index()
# rb2.head()

rb2_ts.rename(columns={'ID':'ID',
            'DATE-REPORTED':'Total_Times_reported',
            'DISBURSED-DT':'Total_Times_disbursed',
            'CLOSE-DT':'Total_Times_closed',
            'LAST-PAYMENT-DATE':'Total_last_payments'},inplace=True)

bureua_data_f_ts =pd.merge(bureua_data_f_ts,rb2_ts,how='inner',on='ID',)
bureua_data_f_ts.shape

(14745, 94)

In [23]:
df_test_b['REPORTED DATE - HIST']  = df_test_b['REPORTED DATE - HIST'].apply(lambda x: x.split(',') if type(x)==str else x).apply(lambda x: x[:-1] if type(x)==list else x)
df_test_b['REPORTED DATE - HIST'] = df_test_b['REPORTED DATE - HIST'].apply(lambda x: len(x) if type(x)==list else x)
rc1_ts = df_test_b.groupby('ID')['REPORTED DATE - HIST'].sum().reset_index()

rc1_ts.columns = ['ID','Total_Times_Reported']
bureua_data_f_ts =pd.merge(bureua_data_f_ts,rc1_ts,how='inner',on='ID',)
bureua_data_f_ts.shape

(14745, 95)

In [24]:
def amt_sum(clm,df):
    for cl in clm:
        df[cl] = df[cl].apply(lambda x: list(filter(lambda item: item,x.split(','))) if type(x)==str else x) #remove comma
        df[cl] = df[cl].apply(lambda d: d if isinstance(d, list) else [0.0])#fill na
        df[cl] = df[cl].apply(lambda x: list(map(float,x)) if type(x)==list else x)#convert all the list elements to float
        df[cl] = df[cl].apply(lambda x:sum(x))#summ the ammount

amt_sm_cl = ['CUR BAL - HIST','AMT OVERDUE - HIST','AMT PAID - HIST']
amt_sum(amt_sm_cl,df_test_b)

rd1_ts = df_test_b.groupby('ID')[amt_sm_cl].sum().reset_index()
rd1_ts.rename(columns={'ID':'ID','CUR BAL - HIST':'Total_CUR_BAL','AMT OVERDUE - HIST':'Total_Overdue_Amt','AMT PAID - HIST':'Total_Paid_Amt'})

bureua_data_f_ts =pd.merge(bureua_data_f_ts,rd1_ts,how='inner',on='ID',)
bureua_data_f_ts.shape

(14745, 98)

In [25]:
import textwrap
def for_scN_DPD(nu):
    if 'E' in nu:
        nu = '%f'%np.float64(nu)
        nu = textwrap.wrap(nu,3)
        # print(nu)
        nu =list(map(float,nu))

    return sum(nu)

#split into part of 3 digits
#check if its string
#if its not filter out keep only ints
#sum the total of list
def clean_DPD(x):
    wp = textwrap.wrap(x,3)
    if not all(str(s).isdigit() for s in wp):
        # print(wp
        wp = list(filter(lambda x: x not in ['XXX','DDD'], wp))
        wp = list(map(int, wp))
    else:
        wp = list(map(int,wp))

    return sum(wp)

# for_scN_DPD(df_train_b['DPD - HIST'][26])
df_test_b['DPD - HIST'].fillna('000',inplace=True)
df_test_b['DPD - HIST'] = df_test_b['DPD - HIST'].apply(lambda x:for_scN_DPD(x) if 'E' in x else clean_DPD(x))
rd3_ts = df_test_b.groupby('ID')['DPD - HIST'].sum().reset_index()

rd3_ts.columns = ['ID','Total_DPD']
bureua_data_f_ts =pd.merge(bureua_data_f_ts,rd3_ts,how='inner',on='ID',)
bureua_data_f_ts.shape

(14745, 99)

In [26]:
print(len(result_a2_ts.columns))
def drop_VC_BL_100(clm_nm,data_f):
    for i in df_test_b[clm_nm].value_counts().loc[lambda x : x<100].index:
        data_f.drop(clm_nm+'_%s'%i,axis=1,inplace=True)

#now for CONTRIBUTOR-TYPE_ ACCOUNT-STATUS_ ASSET_CLASS_
# for cl in ['ACCT-TYPE','CONTRIBUTOR-TYPE','ACCOUNT-STATUS','ASSET_CLASS']:
#     drop_VC_BL_100(cl,result_a2)

print(len(result_a2_ts.columns))

for cl in ['ACCT-TYPE','CONTRIBUTOR-TYPE','ACCOUNT-STATUS','ASSET_CLASS']:
    drop_VC_BL_100(cl,bureua_data_f_ts)

83
83


In [27]:
def mss_ct(ref_cl,ms_cl,df):
    ''' function to fill the NaN values by mode method
        Ex: in Particular State which is more frequent occured city
            same for Area in Area
                     SEX in Area
                     Age in Area
                     Salary in Area
        ref_cl : Referece Column,
        ref_nm : Element of refernce column, 
        df     : dataframe to oerform fill na operation
    
    '''
    md = df[ms_cl].mode()[0]
    frqnt_val = lambda x: x.fillna(x.value_counts().idxmax() if x.value_counts().max() >=1 else md , inplace = False)
    df[ms_cl]=df.groupby([ref_cl])[ms_cl].apply(frqnt_val)
    df[ms_cl] = df[ms_cl].fillna(df[ms_cl].value_counts().idxmax())
    # return df

In [29]:
# def find_VC_St(ref_cl,ref_nm,tg):
#     ''' ref_cl : Referece Column,
#         ref_nm : Element of refernce column,
#         tg     : column name to fill nan   '''
#     vl_cm =df_test_data[df_test_data[ref_cl]==ref_nm][tg].value_counts()
# #     print(vl_cm)
#     re_nm_tg_cnt = (vl_cm.index[0],vl_cm[0])
#     return re_nm_tg_cnt

# print(find_VC_St('State','WEST BENGAL','City'),find_VC_St('State','ANDHRA PRADESH','City'))

mss_ct('State','City',df_test_data)
mss_ct('City','Area',df_test_data)
mss_ct('Area','SEX',df_test_data)
mss_ct('Area','AGE',df_test_data)
mss_ct('Area','MonthlyIncome',df_test_data)

# print(find_VC_St('State','WEST BENGAL','City'),find_VC_St('State','ANDHRA PRADESH','City'))
df_test_data['MaturityDAte'].fillna(method='ffill',inplace=True)
print(any(df_test_data.isna().sum()>0))

True


In [30]:
df_test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14745 entries, 0 to 14744
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ID               14745 non-null  int64         
 1   Frequency        14745 non-null  float64       
 2   InstlmentMode    14745 non-null  float64       
 3   LoanStatus       14745 non-null  float64       
 4   PaymentMode      14745 non-null  float64       
 5   BranchID         14745 non-null  int64         
 6   Area             14745 non-null  float64       
 7   Tenure           14745 non-null  int64         
 8   AssetCost        14745 non-null  int64         
 9   AmountFinance    14745 non-null  float64       
 10  DisbursalAmount  14745 non-null  float64       
 11  EMI              14745 non-null  float64       
 12  DisbursalDate    14745 non-null  datetime64[ns]
 13  MaturityDAte     14745 non-null  datetime64[ns]
 14  AuthDate         14745 non-null  datet

In [31]:
dro_clm = ['BranchID','AssetID','ManufacturerID','SupplierID','ZiPCODE']
df_test_data.drop(dro_clm,axis=1,inplace=True)

dt_cl = ['DisbursalDate','MaturityDAte','AuthDate']

def handel_dt_ts(clm):
    for cl in clm:
        df_test_data[cl+'_Year'] = df_test_data[cl].dt.year.astype(float)
        df_test_data[cl+'_Month']= df_test_data[cl].dt.month.astype(float)
        df_test_data[cl+'_Week']= df_test_data[cl].dt.week.astype(float)
        df_test_data[cl+'_Day']= df_test_data[cl].dt.day.astype(float)

handel_dt_ts(dt_cl)
df_test_data.drop(dt_cl,axis=1,inplace=True)

c:\users\vishal.nadagiri\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  # Remove the CWD from sys.path while we load stuff.


In [32]:
test_id = df_test_data['ID']
target = df_train_data['Top-up Month']

TUP_Mn = { id_en:i for i,id_en in enumerate(df_train_data['Top-up Month'].value_counts().index) }

target = target.map(TUP_Mn)

In [33]:
print('Before droping na',df_test_data.shape)
df_test_data.dropna(inplace=True)
print('After droping na',df_test_data.shape)

Before droping na (14745, 29)
After droping na (14745, 29)


In [35]:
all_test = pd.merge(bureua_data_f_ts,df_test_data,how='inner',on='ID',)

print('Before droping na',all_train.shape)
all_test.dropna(inplace=True)
print('After droping na',all_train.shape)
all_test.drop(['ID'],axis=1,inplace=True)

Before droping na (128655, 112)
After droping na (128655, 112)


In [40]:
all_train.shape

(128655, 112)

In [41]:
all_test.shape

(14745, 94)

In [46]:
for trn in all_train.columns:
    if trn not in all_test.columns:
#         print(trn)
        all_train.drop([trn],axis=1,inplace=True)

In [47]:
from sklearn.ensemble import RandomForestClassifier

In [67]:
model_rstne_100 = RandomForestClassifier(100, oob_score=True, n_jobs=-1, random_state=101)
model_rstne_100.fit(all_train, target)
y_pred_rftne_100 = model_rstne_100.predict(all_test)

In [68]:
sb_sp =pd.read_csv('sample_submission_ejm25Dc.csv')

In [69]:
sb_sp.columns[1]

'Top-up Month'

In [70]:
TUP_Mn = { id_en:i for i,id_en in enumerate(df_train_data['Top-up Month'].value_counts().index) }

In [71]:
target.value_counts()

0    106677
1      8366
2      3656
3      3492
4      3062
5      2368
6      1034
Name: Top-up Month, dtype: int64

In [72]:
df_train_data['Top-up Month'].value_counts()

No Top-up Service    106677
 > 48 Months           8366
36-48 Months           3656
24-30 Months           3492
30-36 Months           3062
18-24 Months           2368
12-18 Months           1034
Name: Top-up Month, dtype: int64

In [73]:
pd.Series(y_pred_rftne_100).value_counts()

0    14392
1      331
2       11
3        6
5        3
4        2
dtype: int64

In [74]:
TUP_Mn

{'No Top-up Service': 0,
 ' > 48 Months': 1,
 '36-48 Months': 2,
 '24-30 Months': 3,
 '30-36 Months': 4,
 '18-24 Months': 5,
 '12-18 Months': 6}

In [75]:
test_map = {0:'No Top-up Service',
            1:' > 48 Months',
            2:'36-48 Months',
            3:'24-30 Months',
            4:'30-36 Months',
            5:'18-24 Months'
           }


In [76]:
pd.Series(y_pred_rftne_100).map(test_map).value_counts()

No Top-up Service    14392
 > 48 Months           331
36-48 Months            11
24-30 Months             6
18-24 Months             3
30-36 Months             2
dtype: int64

In [78]:
y_pred_rftne_dum = pd.Series(y_pred_rftne_100).map(test_map)
y_pred_rftne_dum.value_counts()

No Top-up Service    14392
 > 48 Months           331
36-48 Months            11
24-30 Months             6
18-24 Months             3
30-36 Months             2
dtype: int64

In [79]:
y_pred_rftne_100 = pd.Series(y_pred_rftne_100).map(test_map)

In [80]:
y_pred_rftne_100.value_counts()

No Top-up Service    14392
 > 48 Months           331
36-48 Months            11
24-30 Months             6
18-24 Months             3
30-36 Months             2
dtype: int64

In [82]:
to_csv = pd.DataFrame({sb_sp.columns[0]:test_id,
                       sb_sp.columns[1]:y_pred_rftne_100})

In [83]:
to_csv.head()

,ID,Top-up Month
0,4,No Top-up Service
1,5,No Top-up Service
2,6,No Top-up Service
3,25,No Top-up Service
4,119,No Top-up Service


In [84]:
to_csv.shape

(14745, 2)

In [85]:
all_test.shape

(14745, 94)

In [88]:
to_csv.to_csv('submission_1.csv',index=False)

In [92]:
# !pip install pickle-mixin
import pickle

In [95]:
saved_md = pickle.load(open('model_rstne_100.sav', 'rb'))

c:\users\vishal.nadagiri\appdata\local\programs\python\python36\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.2.post1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
y_svd = saved_md.predict(all_test)

y_svd_100 = pd.Series(y_svd).map(test_map)

to_csv = pd.DataFrame({sb_sp.columns[0]:test_id,
                       sb_sp.columns[1]:y_pred_rftne_100})

to_csv.to_csv('submission_1.csv',index=False)